# Example Notebook to showcasing how we interact with JTReaders

## Prerequisites

Download GloVe
> `data/GloVe/download_small.sh`

In [1]:
# First change dir to JTR parent
import os
os.chdir('..')

from jack.io.embeddings.embeddings import load_embeddings
from jack.util.vocab import Vocab
from jack.core import SharedResources
from jack.io.load import load_jack
from jack.readers as readers
from jack.util.hooks import LossHook, ExamplesPerSecHook
from jack.io.load import load_jack
import tensorflow as tf

A list of all currently available readers from `readers.py`

In [2]:
print(", ".join(readers.readers.keys()))

fastqa_reader, modular_qa_reader, fastqa_reader_torch, dam_snli_reader, cbilstm_nli_reader, modular_nli_reader, distmult_reader, complex_reader, transe_reader


## Shared resources

In [3]:
# we need a vocabulary (with embeddings for our fastqa_reader, but this is not always necessary)
embeddings = load_embeddings('data/GloVe/glove.6B.50d.txt', 'glove')
vocab = Vocab(emb=embeddings,
              init_from_embeddings=True)

## FastQA (SQuAD)

### Data loading

Load up the training data

In [4]:
fastqa_train_data = load_jack('data/SQuAD/snippet.jtr.json')

### Creating the reader

TODO
Model configuration
+
Where to find more details

In [5]:
fastqa_config = {"repr_dim": 10,
                 "repr_dim_input": embeddings.lookup.shape[1],
                 "max_span_size": 10}

Create an example reader, based on vocabulary and the reader configuration

In [6]:
fastqa_svac = SharedResources(vocab, fastqa_config)
fastqa_reader = readers.fastqa_reader(fastqa_svac)

TODO: Explain what setup form data does

In [7]:
# all parameters are initialized after this call
fastqa_reader.setup_from_data(fastqa_train_data, is_training=True)

### Applying the untrained reader

In [8]:
# take a list of inputs, e.g., from our training data
questions = [q for q, a in fastqa_train_data]
for q, a in zip(questions[:5], fastqa_reader(questions)[:5]):
    print("Question: " + q.question)
    print("Answer:   %s \t %.3f" % (a[0].text, a[0].score))
    print()

Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer:   legend 	 -0.142

Question: What is in front of the Notre Dame Main Building?
Answer:   legend 	 -0.165

Question: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
Answer:   legend 	 -0.162

Question: What is the Grotto at Notre Dame?
Answer:   legend 	 -0.147

Question: What sits on top of the Main Building at Notre Dame?
Answer:   legend 	 -0.169



Of course the output is not correct because the model was not trained at all

### Training

Train setup

In [9]:
# for training we use the bin/jack-train.py script.
batch_size = len(fastqa_train_data)
# short explanation
hooks = [LossHook(fastqa_reader, iter_interval=1), 
         ExamplesPerSecHook(fastqa_reader, batch_size, iter_interval=1)]
optimizer = tf.train.AdamOptimizer(0.11)

...and the training itself:

In [10]:
fastqa_reader.train(optimizer,
                    batch_size=batch_size,
                    hooks=hooks,
                    max_epochs=20,
                    training_set=fastqa_train_data)

/Users/matko/workspace/jack/venv/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:jack.core.reader:Number of parameters: 6341
INFO:jack.core.reader:Start training...
INFO:jack.util.hooks:Epoch 1	Iter 1	train loss 10.043832778930664
INFO:jack.util.hooks:Epoch 2	Iter 2	train loss 10.1898832321167
INFO:jack.util.hooks:Epoch 3	Iter 3	train loss 8.506146430969238
INFO:jack.util.hooks:Epoch 4	Iter 4	train loss 7.6635918617248535
INFO:jack.util.hooks:Epoch 5	Iter 5	train loss 7.650871276855469
INFO:jack.util.hooks:Epoch 6	Iter 6	train loss 7.5434746742248535
INFO:jack.util.hooks:Epoch 7	Iter 7	train loss 6.123935699462891
INFO:jack.util.hooks:Epoch 8	Iter 8	train loss 5.673914432525635
INFO:jack.util.hooks:Epoch 9	Iter 9	train loss 5.2398176193237305
INFO:jack.util.hooks:Epoch 10	Iter 10	train loss 4.7373881340026855
INFO:jack.util.hooks:Epoch 11	Iter 11	train loss 3.9916515350341797
INFO:jack.util.hooks:Epoch 12	Iter 12	train loss 3.6511764526367188
INFO:jack.util.hooks:Epoch 13	Iter 13	train loss 3.220385789871216
INFO:jack.util.hooks:Epoch 14	Iter 14	train loss 4.3

### Predictions from the trained reader

In [11]:
# After training the output should look better
for q, a in zip(questions[:5], fastqa_reader(questions)[:5]):
    print("Question: " + q.question)
    print("Answer:   %s \t (score: %.3f)\n" % (a[0].text, a[0].score))

Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer:   a copper statue of Christ 	 (score: -0.888)

Question: What is in front of the Notre Dame Main Building?
Answer:   a copper statue of Christ 	 (score: -0.709)

Question: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
Answer:   a copper statue of Christ 	 (score: -0.985)

Question: What is the Grotto at Notre Dame?
Answer:   a copper statue of Christ 	 (score: -0.987)

Question: What sits on top of the Main Building at Notre Dame?
Answer:   a copper statue of Christ 	 (score: -0.912)



### Saving the model

In [12]:
# saving into the dir ()
fastqa_reader.store("/tmp/fastqa_reader")

## Decomposable attention model (SNLI)

### Data loading

In [13]:
snli_train_data = load_jack('data/SNLI/snippet.jtr_v1.json')

hypotheses = []
premises = []
labels = []
for input_, output_ in snli_train_data:
    premises.append(input_.support[0])
    hypotheses.append(input_.question)
    labels.append(output_[0].text)

### Creating the reader

In [14]:
snli_config = {"repr_dim": 10,
               "repr_dim_input": embeddings.lookup.shape[1],
               "model": "dam_snli_reader"}

In [15]:
# create example reader
snli_svac = SharedResources(vocab, snli_config)

In [21]:
# for training we use the bin/jack-train.py script.
# print("\x1b[31m\"red\"\x1b[0m")However, programatically we could quickly train a model like this

# setup reader ini training mode
tf.reset_default_graph()
snli_reader = readers.readers["dam_snli_reader"](snli_svac)
snli_reader.setup_from_data(snli_train_data, is_training=True)

INFO:jack.readers.natural_language_inference.decomposable_attention:Building the Attend graph ..
INFO:jack.readers.natural_language_inference.decomposable_attention:Building the Compare graph ..
INFO:jack.readers.natural_language_inference.decomposable_attention:Building the Aggregate graph ..


### Training

Train setup

In [22]:
batch_size = len(snli_train_data)
hooks = [LossHook(snli_reader, iter_interval=1), 
         ExamplesPerSecHook(snli_reader, batch_size, iter_interval=1)]
optimizer = tf.train.AdamOptimizer(0.05)

...and the training:

In [23]:
snli_reader.train(optimizer,
                  batch_size=batch_size,
                  hooks=hooks,
                  max_epochs=20,
                  training_set=snli_train_data)

INFO:jack.core.reader:Preparing training data...
INFO:jack.core.input_module:OnlineInputModule pre-processes data on-the-fly in first epoch and caches results for subsequent epochs! That means, first epoch might be slower.
INFO:jack.core.reader:Number of parameters: 20001443
INFO:jack.core.reader:Start training...
INFO:jack.util.hooks:Epoch 1	Iter 1	train loss 1.0986123085021973
INFO:jack.util.hooks:Epoch 2	Iter 2	train loss 1.0929327011108398
INFO:jack.util.hooks:Epoch 3	Iter 3	train loss 1.0885658264160156
INFO:jack.util.hooks:Epoch 4	Iter 4	train loss 1.0846277475357056
INFO:jack.util.hooks:Epoch 5	Iter 5	train loss 1.0950920581817627
INFO:jack.util.hooks:Epoch 6	Iter 6	train loss 1.073133111000061
INFO:jack.util.hooks:Epoch 7	Iter 7	train loss 1.063262701034546
INFO:jack.util.hooks:Epoch 8	Iter 8	train loss 1.0115700960159302
INFO:jack.util.hooks:Epoch 9	Iter 9	train loss 0.9028619527816772
INFO:jack.util.hooks:Epoch 10	Iter 10	train loss 0.8432081937789917
INFO:jack.util.hooks:Epo

### Predictions from the trained reader

In [24]:
input_ = [qa_setting for qa_setting, answers in snli_train_data]
output_ = snli_reader(input_)

In [25]:
for p, h, l, o in zip(premises, hypotheses, labels, output_):
    print('Premise: {}'.format(p))
    print('Hypothesis: {}'.format(h))
    print('Prediction: {} (score: {:.2f})  [Label: {}]\n'.format(o[0].text, o[0].score, l))

Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is training his horse for a competition.
Prediction: neutral (score: 36.41)  [Label: neutral]

Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is at a diner, ordering an omelette.
Prediction: entailment (score: 0.45)  [Label: contradiction]

Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is outdoors, on a horse.
Prediction: entailment (score: 0.45)  [Label: entailment]

Premise: Children smiling and waving at camera
Hypothesis: They are smiling at their parents
Prediction: neutral (score: 30.91)  [Label: neutral]

Premise: Children smiling and waving at camera
Hypothesis: There are children present
Prediction: entailment (score: 0.45)  [Label: entailment]

Premise: Children smiling and waving at camera
Hypothesis: The kids are frowning
Prediction: entailment (score: 0.45)  [Label: contradiction]

Premise: A boy is jumping on ska